In [1]:
function f()
    local i
    for outer i in 1:10^6
        i > 5*10^5 && break
    end
    i
end

f (generic function with 1 method)

In [2]:
function g()
    i = undef
    for outer i in 1:10^6
        i > 5*10^5 && break
    end
    i
end

g (generic function with 1 method)

In [3]:
f()

500001

In [4]:
@code_warntype f()

Variables
  #self#::Core.Const(f)
  @_2::Union{Nothing, Tuple{Int64, Int64}}
  i::Int64

Body::Int64
1 ─       Core.NewvarNode(:(i))
│   %2  = Core.apply_type(Base.Val, 6)::Core.Const(Val{6})
│   %3  = (%2)()::Core.Const(Val{6}())
│   %4  = Base.literal_pow(Main.:^, 10, %3)::Int64
│   %5  = (1:%4)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_2 = Base.iterate(%5))
│   %7  = (@_2 === nothing)::Bool
│   %8  = Base.not_int(%7)::Bool
└──       goto #6 if not %8
2 ┄ %10 = @_2::Tuple{Int64, Int64}::Tuple{Int64, Int64}
│         (i = Core.getfield(%10, 1))
│   %12 = Core.getfield(%10, 2)::Int64
│   %13 = i::Int64
│   %14 = Core.apply_type(Base.Val, 5)::Core.Const(Val{5})
│   %15 = (%14)()::Core.Const(Val{5}())
│   %16 = Base.literal_pow(Main.:^, 10, %15)::Int64
│   %17 = (5 * %16)::Int64
│   %18 = (%13 > %17)::Bool
└──       goto #4 if not %18
3 ─       goto #6
4 ─       (@_2 = Base.iterate(%5, %12))
│   %22 = (@_2 === nothing)::Bool
│   %23 = Base.not_int(%22)

In [5]:
@code_warntype g()

Variables
  #self#::Core.Const(g)
  @_2::Union{Nothing, Tuple{Int64, Int64}}
  i::Union{UndefInitializer, Int64}

Body::Union{UndefInitializer, Int64}
1 ─       (i = Main.undef)
│   %2  = Core.apply_type(Base.Val, 6)::Core.Const(Val{6})
│   %3  = (%2)()::Core.Const(Val{6}())
│   %4  = Base.literal_pow(Main.:^, 10, %3)::Int64
│   %5  = (1:%4)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_2 = Base.iterate(%5))
│   %7  = (@_2 === nothing)::Bool
│   %8  = Base.not_int(%7)::Bool
└──       goto #6 if not %8
2 ┄ %10 = @_2::Tuple{Int64, Int64}::Tuple{Int64, Int64}
│         (i = Core.getfield(%10, 1))
│   %12 = Core.getfield(%10, 2)::Int64
│   %13 = i::Int64::Int64
│   %14 = Core.apply_type(Base.Val, 5)::Core.Const(Val{5})
│   %15 = (%14)()::Core.Const(Val{5}())
│   %16 = Base.literal_pow(Main.:^, 10, %15)::Int64
│   %17 = (5 * %16)::Int64
│   %18 = (%13 > %17)::Bool
└──       goto #4 if not %18
3 ─       goto #6
4 ─       (@_2 = Base.iterate(%5, %12))
│   %22 =

In [6]:
using BenchmarkTools

In [7]:
@btime f()

  381.300 μs (0 allocations: 0 bytes)


500001

In [8]:
@btime g()

  357.300 μs (0 allocations: 0 bytes)


500001

In [9]:
@code_native debuginfo=:none f()

	.text
	pushq	%rbp
	movq	%rsp, %rbp
	subq	$32, %rsp
	movabsq	$power_by_squaring, %rax
	movl	$10, %ecx
	movl	$6, %edx
	callq	*%rax
	testq	%rax, %rax
	jle	L119
	movq	%rax, %rcx
	sarq	$63, %rcx
	andnq	%rax, %rcx, %r8
	movl	$1, %eax
	nopw	%cs:(%rax,%rax)
L64:
	movl	$10, %edx
	movl	$2, %ecx
	nopw	(%rax,%rax)
L80:
	imulq	%rdx, %rdx
	decq	%rcx
	jg	L80
	imulq	$50, %rdx, %rcx
	cmpq	%rax, %rcx
	jl	L113
	leaq	1(%rax), %rcx
	cmpq	%r8, %rax
	movq	%rcx, %rax
	jne	L64
	movq	%r8, %rax
L113:
	addq	$32, %rsp
	popq	%rbp
	retq
L119:
	movabsq	$jl_undefined_var_error, %rax
	movl	$518228608, %ecx                # imm = 0x1EE38A80
	callq	*%rax
	nopl	(%rax,%rax)


In [10]:
@code_native debuginfo=:none g()

	.text
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%rsi
	pushq	%rdi
	subq	$32, %rsp
	movq	%rcx, %rdi
	movabsq	$power_by_squaring, %rax
	movl	$10, %ecx
	movl	$6, %edx
	callq	*%rax
	testq	%rax, %rax
	jle	L120
	movq	%rax, %rcx
	sarq	$63, %rcx
	andnq	%rax, %rcx, %rcx
	movl	$1, %eax
	nopl	(%rax)
L64:
	movl	$10, %edx
	movl	$2, %esi
	nopw	(%rax,%rax)
L80:
	imulq	%rdx, %rdx
	decq	%rsi
	jg	L80
	imulq	$50, %rdx, %rdx
	cmpq	%rax, %rdx
	jl	L113
	leaq	1(%rax), %rdx
	cmpq	%rcx, %rax
	movq	%rdx, %rax
	jne	L64
	movq	%rcx, %rax
L113:
	movq	%rax, (%rdi)
	movb	$2, %dl
	jmp	L122
L120:
	movb	$1, %dl
L122:
	xorl	%eax, %eax
	addq	$32, %rsp
	popq	%rdi
	popq	%rsi
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)
